In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_store_mch4_monthly (
  calmonth DATE,
  store_id STRING,
  mch4_id STRING,
  rev_amt DOUBLE,
  rev_amt_pos DOUBLE,
  rev_promotion_discount DOUBLE,
  closing_stock_value DOUBLE,
  closing_stock_quantity DOUBLE,
  no_product_had_sale BIGINT,
  rev_amt_pos_mdl DOUBLE,
  billcount BIGINT,
  closing_stock_value_dio DOUBLE,
  cogs_b2b_30d DOUBLE,
  no_product_active BIGINT)
USING delta
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
);
""")

In [0]:
spark.sql(f"""DELETE FROM {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_store_mch4_monthly
WHERE calmonth >= DATE(DATE_TRUNC('month', CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - INTERVAL 1 MONTH)""")

In [0]:
spark.sql(f""" INSERT INTO {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_store_mch4_monthly
WITH cte AS (
  SELECT t.calmonth,
      a.store_id,
      product.mch4_id,
      SUM(a.revenue.billcnt_total) billcnt_total,
      SUM(a.revenue.rev_amt) rev_amt,
      SUM(a.revenue.rev_amt_pos) rev_amt_pos,
      SUM(a.revenue.rev_promo_amt) rev_promotion_discount,
      SUM(CASE WHEN a.calday = t.calsunday THEN a.stock.closing_stock_value END) closing_stock_value,
      SUM(CASE WHEN a.calday = t.calsunday THEN a.stock.closing_stock_quantity END) closing_stock_quantity,
      COUNT(DISTINCT CASE WHEN a.revenue.billcnt_total > 0 THEN a.product_id END) no_product_had_sale,
      SUM(CASE WHEN product.manufacturer_id IN ('2325', '1915') THEN a.revenue.rev_amt_pos ELSE 0 END) rev_amt_pos_mdl
  FROM (SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
        WHERE calday >= DATE(DATE_TRUNC('month', CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - INTERVAL 1 MONTH)) a
   LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time t ON a.calday = t.calday
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product product ON a.product_id = product.product_id
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store store ON a.store_id = store.store_id
  WHERE product.mch2_id IN ('101', '102', '201', '202', '203')
    AND store.business_unit IN ('1500')
    AND store.store_id NOT LIKE '195%'
    AND product.mch4_id NOT IN ('1010104', '1010208')
  GROUP BY 1, 2, 3
  HAVING rev_amt_pos <> 0
    OR billcnt_total > 0
    OR closing_stock_value <> 0
    OR closing_stock_quantity <> 0
),
a_daily_store_mch4_data AS (
  SELECT t.calmonth,
    a.store_id,
    a.mch4_id,
    SUM(billcount) billcount  FROM {catalog_name}.udp_wcm_gold_vcm_finance_self_service.a_daily_store_mch4_data a
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time t ON a.day = t.calday
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store store ON a.store_id = store.store_id
  WHERE store.business_unit IN ('1500')
    AND t.calday >= DATE(DATE_TRUNC('month', CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - INTERVAL 1 MONTH)
  GROUP BY 1, 2, 3
),
dio AS (
  SELECT b.calmonth,
         a.store_id,
         prd.mch4_id,
         SUM(a.closing_stock_value) closing_stock_value_dio,
         SUM(a.cogs_b2b_30d) cogs_b2b_30d
  FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_dr_dio_rp_store_sku a
  LEFT JOIN (SELECT DISTINCT mch4_id, product_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product) prd ON a.product_id = prd.product_id
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time b ON a.calday = b.calday
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
  WHERE a.calday >= DATE(DATE_TRUNC('month', CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - INTERVAL 1 MONTH)
    AND c.business_unit = '1500'
    AND a.store_id NOT IN ('1950', '1951', '1952')
    AND mch4_id NOT IN ('1010104', '1010208')
    AND a.dio_type = 1
  GROUP BY 1, 2, 3
),
assortment AS (
  SELECT t.calmonth, a.store_id, product.mch4_id,
    COUNT(DISTINCT a.product_id) no_product_active
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his a
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_time t ON a.calday = t.calday
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product product ON a.product_id = product.product_id
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store store ON a.store_id = store.store_id
  WHERE product.mch2_id IN ('101', '102', '201', '202', '203')
    AND store.business_unit = '1500'
    AND store.store_id NOT IN ('1950', '1951', '1952')
    AND product.mch4_id NOT IN ('1010104', '1010208')
    AND a.calday >= DATE(DATE_TRUNC('month', CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) - INTERVAL 1 MONTH)
  GROUP BY 1, 2, 3
)
SELECT cte.* EXCEPT(billcnt_total),
      a_daily_store_mch4_data.billcount,
      dio.closing_stock_value_dio,
      dio.cogs_b2b_30d,
      assortment.no_product_active
FROM cte
LEFT JOIN a_daily_store_mch4_data USING(calmonth, store_id, mch4_id)
LEFT JOIN dio USING(calmonth, store_id, mch4_id)
LEFT JOIN assortment USING(calmonth, store_id, mch4_id)
""")